# Valutazione del dataset

Verrà eseguita la valutazione sia del dataset senza PE, sia con quello PE.

In [ ]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import spacy

In [ ]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

sentences_en = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_en.csv', header=None, delimiter='aaaaaaaa', names=[0])

i = 0

for sentence in sentences_en[0]:
    current_lid = ""
    current_en_sentence = ""
    i += 1
    print(i)

    for dataset_type in dataset_types:
        if current_lid == "":
            for triple_number in triple_numbers:
                path = "..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
                file_names = []

                for file_name in os.listdir(path):
                    if os.path.isfile(os.path.join(path, file_name)):
                        url = os.path.join(path, file_name)
                        tree = ET.parse(url)
                        root = tree.getroot()

                        for entry in root.iter('entry'):
                            if current_lid == "":
                                for lex in entry.iter('lex'):
                                    if lex.get('lang') == 'en':
                                        if lex.text == sentence:
                                            current_lid = lex.attrib['lid']
                                            current_en_sentence = lex.text

                                if current_lid != "":
                                    for lex in entry.iter('lex'):
                                        if lex.get('lang') == 'it-PE':
                                            if lex.attrib['lid'] == current_lid:
                                                dataset.append(str(i) + " " + lex.text)
                                                print(current_en_sentence)
                                                print(lex.text)
                                                print('\n')
                                                break
                                    

dataset = np.array(dataset)

In [ ]:
dataset_copy = dataset.copy()
#rimuovi i primi 4 caratteri di ogni riga
dataset_copy = np.char.lstrip(dataset_copy, '1234567890 ')

dataset_copy

In [ ]:
dt_dataset = pd.DataFrame(dataset_copy)
dt_dataset.to_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl_PE.csv', index=False, header=False)

## Valutazione

In [ ]:
#utils
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

#blue 
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

#meteor
from nltk.translate.meteor_score import meteor_score

#pvalue
from scipy.stats import ttest_ind

df_manual = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_manual.csv', header=None, delimiter='aaaaaaaa', names=[0])
df_deepl = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl.csv', header=None, delimiter='aaaaaaaa', names=[0])
df_deepl_PE = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl_PE.csv', header=None, delimiter='aaaaaaaa', names=[0])

### BLEU

In [ ]:
def blue_evaluation(df):
    blue_scores = []
    
    for i in range(len(df)):
        reference = df_manual[0][i]
        candidate = df[0][i]

        #remove " from candidate
        candidate = candidate.replace('"', '')

        blue_score = sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
        blue_scores.append(blue_score)
        
    return blue_scores

In [ ]:
print('BLUE SCORE \n')

df_deepl['bleu_score'] = blue_evaluation(df_deepl)
print('- DeepL: ', df_deepl['bleu_score'].mean())

df_deepl_PE['bleu_score'] = blue_evaluation(df_deepl_PE)
print('- DeepL PE: ', df_deepl_PE['bleu_score'].mean())

### METEOR

In [ ]:
def meteor_evaluation(df):
    meteor_scores = []
    
    for i in range(len(df)):
        reference = [word_tokenize(df_manual[0][i])]
        candidate = word_tokenize(df[0][i])

        candidate = [word.replace('"', '') for word in candidate]

        meteor_scores.append(meteor_score(reference, candidate))
        
    return meteor_scores

In [ ]:
print('\nMETEOR SCORE \n')

df_deepl['meteor_score'] = meteor_evaluation(df_deepl)
print('- DeepL: ', df_deepl['meteor_score'].mean())

df_deepl_PE['meteor_score'] = meteor_evaluation(df_deepl_PE)
print('- DeepL PE: ', df_deepl_PE['meteor_score'].mean())
